In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [307]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import os
import re
from collections import Counter

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 200

In [130]:
import re
from bs4 import BeautifulSoup as BS
import requests

import multiprocessing
from multiprocessing import Pool
import json

# Scraping from DFS

In [198]:
def getPageLipsticks(index):
    url = "https://www.dfs.com/tc/hong-kong/beauty-and-fragrances/womens/makeup/lipsticks?q=:dlprelevance&page=%d&text="
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
    }
    r = requests.get(url % index, headers=headers)
    items = BS(r.text, "html.parser").findAll("div", {"class": "item"})
    lipsticks = []
    for item in items:
        brand = item.find("p", {"class": "headline"}).text
        price = item.find("p", {"class": "price"}).text
        series = item.find("p", {"class": "description"}).text

        # Get the page of this specific lipstick
        item_link = "https://www.dfs.com" + item.find("a")['href']
        item_r = requests.get(item_link, headers=headers)
        item_soup = BS(item_r.text, "html.parser")

        color_options = item_soup.find("ul", {"class": "color-options"})
        if color_options:
            for li in color_options.find_all("li"):
                color = li.find("label", {"class": "variant-option__color"})
                if color: 
                    lipsticks.append({
                        'brand': brand,
                        'series': series,
                        'price': price,
                        'color': color.text,
                        'name': li.find("input", {"class": "variant-option__value"}).get('value'),
                        'link': item_link
                    })
        else:
            color = item_soup.find("label", {"class": "variant-selected-color"})
            name = item_soup.find("p", {"class": "selected-description"})
            lipsticks.append({
                'brand': brand,
                'series': series,
                'price': price,
                'color': color.text if color else None,
                'name': name.text if name else None,
                'link': item_link
            })
                
    print(index, ':', len(lipsticks))
    with open("./lipsticks/%d.json" % index, "w") as outfile: 
        outfile.write(json.dumps(lipsticks, indent=2))

In [199]:
indexes = [i for i in range(89)]
with Pool(multiprocessing.cpu_count()) as p:
    p.map(getPageLipsticks, indexes)

0 : 42
1 : 0
32 : 0
33 : 0
34 : 0
35 : 0
36 : 0
37 : 0
20 : 20
38 : 0
24 : 20
21 : 0
39 : 0
18 : 29
25 : 0
40 : 0
8 : 20
26 : 20
42 : 0
19 : 0
44 : 0
41 : 0
14 : 20
9 : 0
27 : 0
43 : 0
46 : 0
45 : 0
48 : 0
15 : 0
50 : 0
4 : 20
52 : 0
6 : 20
54 : 0
47 : 0
30 : 20
56 : 0
49 : 0
58 : 0
51 : 0
5 : 0
53 : 0
7 : 0
55 : 0
60 : 0
12 : 27
31 : 0
57 : 0
64 : 0
62 : 0
59 : 0
66 : 0
70 : 0
68 : 0
72 : 0
61 : 0
74 : 0
13 : 0
76 : 0
69 : 0
63 : 0
67 : 0
78 : 0
65 : 0
71 : 0
73 : 0
82 : 0
75 : 0
80 : 0
77 : 0
84 : 0
79 : 0
86 : 0
88 : 0
83 : 0
81 : 0
85 : 0
87 : 0
28 : 48
29 : 0
2 : 21
10 : 28
3 : 0
11 : 0
22 : 20
16 : 20
23 : 0
17 : 0


In [ ]:
for i in range(89):
    getPageLipsticks(i)

In [325]:
TEXTURES = {
    'BALM': 'Balm Rouge',
    'CREAMY': 'Creamy',
    'PEARLY': 'Pearly',
    'MATTE': 'Matte',
    'SHINE': 'Shine',
    'GLOSSY': 'Glossy',
    'SHEER': 'Sheer',
    'SHATIN': 'Shatin',
    'METAL': 'Metal'
}

# Lancome [有在线试妆](https://www.lancome.com.cn/l2_makeup_lips/)

In [195]:
# # HK
# url = "https://www.lancome.com.hk/zh_HK/%E5%BD%A9%E5%A6%9D/%E5%94%87%E9%83%A8/%E5%94%87%E8%86%8F/"
# headers = {
#     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'
# }
# r = requests.get(url, headers=headers)

# BRAND = 'Lancôme'
# TEXTURE_MAPPING = {
#     'Cream': 'Creamy',
#     'Matte': 'Matte',
#     'Shine': 'Shine',
#     'Rôsy': 'Balm Rouge',
# }
# items = BS(r.text, "html.parser").findAll("div", {"class": "product_tile_wrapper"})
# lipsticks = []

# for item in items:
#     series = item.find("div", {"class": "product_description"}).text.replace('\n', '')
#     series_en = item.find("a", {"class": "product-tile__name"}).get("title")
#     price = item.find("p", {"class": "product_price"}).text.replace('\n', '').replace('HK$', 'HKD ')    
#     item_link = item.find("a", {"class": "product-tile__link"}).get("href")
#     texture = TEXTURE_MAPPING[series_en.split()[-1]]
    
    
#     for swatch in item.find_all("li", {"class": "product_swatch_list_item"}):
#         name = swatch.find("a").get("title")
#         lipsticks.append({
#             'brand': BRAND,
#             'liquid': False,
#             'series': series,
#             'series_en': series_en,
#             'price': price,
#             'texture': texture,
#             'color': swatch.find("span").get("data-color") if name != '224-Pink' else '#9a5b64',
#             'name': name,
#             'link': item_link
#         })
        
# df_hk = pd.DataFrame(lipsticks)

In [251]:
# r_lancome = requests.get("https://www.lancome.com.cn/l2_makeup_lips/", headers=headers)

BRAND = 'Lancôme'
TEXTURE_MAPPING = {
    '哑光': TEXTURES['MATTE'],
    '丝绒': TEXTURES['SHEER'],
    '丝缎': TEXTURES['SHATIN'],
    '丝亮': TEXTURES['CREAMY'],
}
items = BS(r_lancome.text, "html.parser").findAll("div", {"class": "thumbnail"})
lipsticks = []

for item in items:
    series = item.find("p", {"class": "goods-tit"}).findChild().text
    if series == '玫瑰润唇底膏SPF10': continue
    series = series.split('丨')[1]
    series_en = item.find("p", {"class": "goods-introudce"}).findChild().text
    price = item.find("div", {"class": "goods-price"}).findChild().text.replace('\t', '').replace('\n', '')
    item_link = "https://www.lancome.com.cn" + item.find("p", {"class": "goods-tit"}).findChild().get("href")
    
    for swatch in item.find_all("div", {"class": "swiper-slide"}):
        swatch = swatch.findChild()
        name = swatch.get("title")
        if series == '菁纯唇膏':
            texture = TEXTURE_MAPPING[name.split("（")[1].split("）")[0]]
        elif series == '菁纯宝石唇膏':
            texture = 'Creamy'
        elif series == '菁纯小姐唇膏':
            texture = 'Shine'
        else:
            texture = 'Glossy'
        
        lipsticks.append({
            'brand': BRAND,
            'liquid': series == '菁纯漆光水唇釉',
            'series': series,
            'series_en': series_en,
            'price': price,
            'texture': texture,
            'color': swatch.get("style").replace("background: ", "").replace(";", ""),
            'name': name,
            'link': item_link
        })
df_lancome = pd.DataFrame(lipsticks)

# Bobbi Brown [有在线试妆](https://www.bobbibrown.com.cn/products/2342/lip-color)


In [292]:
BRAND = 'Bobbi Brown'
TEXTURE_MAPPING = {
    '纯色奢金莹亮唇膏': TEXTURES['SHINE'],
    '纯色奢金哑光唇膏': TEXTURES['MATTE'],
    '纯色奢金钻光唇膏': TEXTURES['SHINE'],
    '纯色奢金唇膏（特别包装）': TEXTURES['SHINE'],
    '炙恋唇膏': TEXTURES['CREAMY'],
    '至盈呵护润唇膏（唇纹小熨斗）': TEXTURES['BALM'],
    '纯色奢金唇膏': TEXTURES['SHEER'],
    '莹亮滋养唇膏·精油唇膏': TEXTURES['GLOSSY'],
    '悦虹唇膏': TEXTURES['CREAMY'],
    '缤纷唇颊霜': TEXTURES['CREAMY'],
    '艺彩唇颊笔': TEXTURES['MATTE'],
    
    '炙恋唇釉': TEXTURES['GLOSSY'],
    '纯色奢金哑光唇釉': TEXTURES['MATTE'],
    '纯色奢金漆光唇釉': TEXTURES['CREAMY'],
    '丰润莹彩唇釉': TEXTURES['GLOSSY'],
}

lipsticks = []

for url in ["https://www.bobbibrown.com.cn/products/2342/lip-color", "https://www.bobbibrown.com.cn/products/2340/Makeup/Lips/Lip-Gloss"]:
    r_bobbibrown = requests.get(url, headers=headers)
    items = BS(r_bobbibrown.text, "html.parser").findAll("div", {"class": "product-brief"})

    for item in items:
        series = item.find("h3", {"class": "product-brief__sub-line"}).text.replace("芭比波朗", "")
        if series in ['纯色奢金唇膏迷你两支装', '双色唇颊颜彩盘', '至盈呵护润唇膏4号'] or series.endswith('唇蜜'): continue
        series_en = item.find("h3", {"class": "product-brief__header"}).text.replace('\n', '')
        price = item.find("span", {"class": "price"}).text
        item_link = "https://www.bobbibrown.com.cn" + item.find("a", {"class": "product-brief__headline-link"}).get('href')
        texture = TEXTURE_MAPPING[series]

        for swatch in item.find_all("a", {"class": "product-brief-shades__shade"}):
            lipsticks.append({
                'brand': BRAND,
                'liquid': url.endswith('Lip-Gloss'),
                'series': series,
                'series_en': series_en,
                'price': price,
                'texture': texture,
                'color': swatch.get("style").replace("background: ", "").replace(";", ""),
                'name': swatch.get("title"),
                'link': item_link
            })
df_bobbibrown = pd.DataFrame(lipsticks)

In [294]:
df_bobbibrown

,brand,liquid,series,series_en,price,texture,color,name,link
0,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#a03b33,4 Claret 干枫叶,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
1,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#ae291a,9 Supernova 超星色,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
2,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#8d4431,2 Bold Honey 花蜜奶茶,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
3,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#a81412,8 Red Stiletto 红色高跟鞋,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
4,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#f13b30,12 Showstopper 活力橙,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
5,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#bf4c53,3 Trailblazer 裸色先锋,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
6,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#ac5c51,1 Bare Truth 裸色真言,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
7,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#ea2c1e,11 Wild Poppy 狂野罂粟,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
8,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#491c17,14 Night Spell 暗夜魔咒,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19
9,Bobbi Brown,False,纯色奢金莹亮唇膏,Luxe Shine Intense,¥300.00,Shine,#993f3f,5 Passion Flower 热情浆果,https://www.bobbibrown.com.cn/product/2342/69380/luxe-shine-intense/fh19


    * Bobbi Brown
    Hourglass
    Estée Lauder
    SAINT LAURENT
    Dior
    * Lancome
    Giorgio Armani
    Make up for ever
    Nars
    givenchy
    tom ford
    NARS
    MAC
    L'Oréal
    Guerlain
    萝卜丁 Chris L
    植村秀
    Chanel
    
    口红质地：
    Balm Rouge（有色唇膏）
    creamy（奶油 A combo of texture and gloss）
    pearly / frost / shimmer (珠光Pearl or Frosted lipsticks combine the benefits of color pigmentation and medium to high shimmer-and-shine finish.）
    matte / velvet（哑光 A sophisticated velvet finish）
    shine（有亮片 Shine and high-shine lipsticks are infused with tiny particles that reflect and mirror light, for a glamourous effect.）
    glossy（these lipsticks give a shiny, watery finish to your pout, which is awesome for dry and thin lips.）
    sheer（润泽 a bit shine and spark.）
    satin（缎面）
    metal
    
    https://www.popxo.com/2016/10/types-of-lipsticks-matte-creamy-liquid-gloss/
    
    种类：
    唇膏
    唇釉
    

# Estée Lauder

In [346]:
BRAND = 'Estée Lauder 雅诗兰黛'
TEXTURE_MAPPING = {
    '倾慕唇膏魅色系列': TEXTURES['CREAMY'],
    '倾慕哑光唇膏丝绒系列': TEXTURES['MATTE'],
    '金管哑光口红': TEXTURES['MATTE'],
    '绝色魅惑唇膏': TEXTURES['SHATIN'],
    '倾慕晶彩唇膏': TEXTURES['SHINE'],
    '恋爱唇膏': None,
    '倾慕哑光唇膏金属系列': TEXTURES['METAL'],
    
    '倾慕唇膏 420轻奢瑰情（玫瑰豆沙色） 13号色': TEXTURES['CREAMY'],
    '绝色倾慕液态唇膏 420轻奢瑰情（玫瑰豆沙色）': TEXTURES['MATTE'],
    '绝色倾慕唇露 420轻奢瑰情（玫瑰豆沙色）': TEXTURES['GLOSSY'],
    
    '绝色倾慕液态唇膏': None,
}

lipsticks = []

for url in ["https://www.esteelauder.com.cn/products/649/product-catalog", "https://www.esteelauder.com.cn/products/647/product-catalog"]:
    r_estee = requests.get(url, headers=headers)
    items = BS(r_estee.text, "html.parser").findAll("div", {"class": "product_brief"})

    for item in items:
        series = item.find("h4", {"class": "product_brief__sub-header"}).text.replace("雅诗兰黛", "")
        if series in ['绝色倾慕修护唇膏', '绝色倾慕液态唇膏420'] or series.endswith('唇蜜'): continue
        series_en = item.find("h3", {"class": "product_brief__header"}).text
        price = item.find("div", {"class": "product_brief__price"}).text.replace('\n', '')
        item_link = "https://www.esteelauder.com.cn" + item.find("a", {"class": "product_brief__image-container"}).get('href')

        for swatch in item.find_all("li", {"class": "swatches--single"}):
            name = swatch.find("a", {"class": "swatch"}).get("name")
            if name == '绝色倾慕液态唇膏 420轻奢瑰情（玫瑰豆沙色）': continue
            texture = TEXTURE_MAPPING[name] if series == '倾慕唇膏温柔豆沙限定系列' else TEXTURE_MAPPING[series] 
            if not texture:
                name_id = name[:3]
                if name_id in ['120', '200', '220', '300', '310', '320', 
                               '103', '201', '202', '301', '302', '303', '304', '403', '404', '420']:
                    texture = 'Matte'
                elif name_id in ['250', '330', '340', 
                                 '106', '203', '305', '306', '307', '408']:
                    texture = 'Satin'
                elif name_id in ['270']:
                    texture = 'Shine'
                elif name_id in ['109', '204', '310', '311', '409']:
                    texture = 'Metal'
                    
            lipsticks.append({
                'brand': BRAND,
                'liquid': '647' in url or '液态' in series,
                'series': series,
                'series_en': series_en,
                'price': price,
                'texture': texture,
                'color': swatch.find("div", {"class": "swatch--1"}).get("style").replace("background-color:", "").replace(";", ""),
                'name': name,
                'link': item_link
            })
df_estee = pd.DataFrame(lipsticks)
df_estee.drop_duplicates(subset='color', keep='first', inplace=True)

In [349]:
df_estee['series'].value_counts()

倾慕唇膏魅色系列        50
绝色魅惑唇膏          25
绝色倾慕液态唇膏        21
倾慕哑光唇膏丝绒系列      14
恋爱唇膏            10
倾慕哑光唇膏金属系列       5
倾慕晶彩唇膏           3
金管哑光口红           3
倾慕唇膏温柔豆沙限定系列     2
Name: series, dtype: int64

In [355]:
df_estee.groupby('brand').nunique()

,brand,liquid,series,series_en,price,texture,color,name,link
brand,,,,,,,,,
Estée Lauder 雅诗兰黛,1,2,9,8,3,7,133,133,10


# Show

In [5]:
# result = []
# for i in range(2):
#     with open('./lipsticks/%d.json' % i) as f:
#         result = result + json.load(f)

# with open("lipsticks.json", "w") as f:
#     json.dump(result, f, indent=2)

In [47]:
result = None
with open('lipsticks.json') as f:
    result = json.load(f, encoding='utf-8')

In [122]:
# clean data
for row in result:

    if row['brand'] == 'Dior':
        if row['series'].startswith('傲姿唇膏'):
            row['series'] = '傲姿唇膏'
        elif row['series'].startswith('誘惑煥彩潤唇膏'):
            row['series'] = '誘惑煥彩潤唇膏'
        elif row['series'].startswith('烈艷藍金摯紅唇膏'):
            row['series'] = '烈艷藍金摯紅唇膏'
        elif row['series'].startswith('Dior Addict'):
            row['series'] = 'Dior Addict 鏡光誘惑唇膏'
        elif row['series'] == '迪奧癮誘超模巨星唇膏 #976 Be Dior':
            row['series'] = '癮誘超模巨星唇膏'
        
    elif row['brand'] == 'SAINT LAURENT':
        if row['series'].startswith('奢華緞面絲絨唇膏'):
            row['series'] = '奢華緞面絲絨唇膏'
        elif row['series'].startswith('情挑誘光水唇膏'):
            row['series'] = '情挑誘光水唇膏'
        elif row['series'].startswith('聖羅蘭奢華緞面唇膏'):
            row['series'] = '奢華緞面唇膏'
        elif row['series'].startswith('彩蜜唇色漆亮系列'):
            row['series'] = '彩蜜唇色漆亮系列'
        elif row['series'].startswith('聖羅蘭奢華緞面漆光唇釉'):
            row['series'] = '奢華緞面漆光唇釉' 
        elif row['series'].startswith('聖羅蘭情挑誘光水唇膏'):
            row['series'] = '情挑誘光水唇膏'
        elif row['series'].startswith('情挑誘吻豐潤蜜唇膏'):
            row['series'] = '情挑誘吻豐潤蜜唇膏'
        elif row['series'].startswith('迷魅亮彩唇膏'):
            row['series'] = '迷魅亮彩唇膏'
        elif row['series'].startswith('奢華緞面鏡光唇釉 水吻系列'):
            row['series'] = '奢華緞面鏡光唇釉 水吻系列'
        elif row['series'].startswith('聖羅蘭奢華緞面漆光唇釉'):
            row['series'] = '奢華緞面漆光唇釉'
        elif row['series'].startswith('彩蜜唇色水漾系列'):
            row['series'] = '彩蜜唇色水漾系列'
        elif row['series'].startswith('聖羅蘭時尚印記絲絨唇露'):
            row['series'] = '時尚印記絲絨唇露'
        elif row['series'].startswith('聖羅蘭瑩亮純魅液體唇膏'):
            row['series'] = '瑩亮純魅液體唇膏'
        elif row['series'].startswith('時尚印記絲絨唇露'):
            row['series'] = '時尚印記絲絨唇露'
        elif row['series'].startswith('限量絕色唇膏'):
            row['series'] = '限量絕色唇膏'
        
        if row['color'] == '#DB1852':
            row['id'], row['name'] = '201', 'Rouge Splash'
        elif row['color'] == '#BC0B28':
            row['id'] = '1'
        elif row['color'] == '#D51D37':
            row['id'], row['name'] = '13', '正橘色'
        elif row['color'] == '#914458':
            row['id'] = '98'
            
    # not well extracted!!!
    elif row['brand'] == 'Lancôme':
        if row['series'].startswith('蘭蔻絕對完美鑽石霧光唇膏'):
            row['series'] = '絕對完美鑽石霧光唇膏'
        elif row['series'].startswith('漆光水唇露'):
            row['series'] = '漆光水唇露'
        elif row['series'].startswith('蘭蔻菁純柔潤脣膏'):
            row['series'] = '蘭蔻菁純柔潤脣膏 (絲絨) '
        elif row['series'].startswith('瑰麗豐盈唇膏液'):
            row['series'] = '瑰麗豐盈唇膏液'
        elif row['series'].startswith('瑰麗豐盈亮唇蜜'):
            row['series'] = '瑰麗豐盈亮唇蜜'
        elif row['series'].startswith('蘭蔻菁純絲絨霧面唇膏'):
            row['series'] = '菁純絲絨霧面唇膏'
        elif row['series'].startswith('蘭蔻菁純寶石唇膏'):
            row['series'] = '菁純寶石唇膏'
            
    elif row['brand'] == 'Bobbi Brown':
        if row['series'].startswith('波朗炙戀唇膏'):
            row['series'] = '炙戀唇膏'
        elif row['series'].startswith('炙戀唇膏'):
            row['series'] = '炙戀唇膏'
        elif row['series'].startswith('至盈呵護潤唇膏（唇紋小熨斗）'):
            row['series'] = '至盈呵護潤唇膏（唇紋小熨斗）'
        elif row['series'].startswith('純色奢金啞光唇膏'):
            row['series'] = '純色奢金啞光唇膏'
        elif row['series'].startswith('藝彩唇頰筆'):
            row['series'] = '藝彩唇頰筆'
        elif row['series'].startswith('純色奢金唇膏'):
            row['series'] = '純色奢金唇膏'
        elif row['series'].startswith('瑩亮滋養唇膏·精油唇膏'):
            row['series'] = '瑩亮滋養唇膏·精油唇膏'
        elif row['series'].startswith('金致奢華唇膏'):
            row['series'] = '金致奢華唇膏'
        elif row['series'].startswith('純色奢金漆光唇釉'):
            row['series'] = '純色奢金漆光唇釉'
        elif row['series'].startswith('限量版純色奢金唇膏'):
            row['series'] = '限量版純色奢金唇膏'
        elif row['series'].startswith('炙戀唇釉'):
            row['series'] = '炙戀唇釉'
        elif row['series'].startswith('悅虹唇膏'):
            row['series'] = '悅虹唇膏'
        elif row['series'].startswith('Bobbi Brown Art Stick Liquid Lip'):
            row['series'] = 'Art Stick Liquid Lip'
        elif row['series'].startswith('Bobbi Brown 藝術家柔霧唇釉筆'):
            row['series'] = '藝術家柔霧唇釉筆'
        elif row['series'].startswith('藝術家柔霧唇釉筆'):
            row['series'] = '藝術家柔霧唇釉筆'
        elif row['series'].startswith('Art Stick Liquid Lip'):
            row['series'] = 'Art Stick Liquid Lip'
            
        row['id'] = row['id'].replace('號', '').replace('#', '')
        if row['color'] == '#D07876':
            row['id'], row['name'] = '2', 'Bare'
        elif row['color'] == '#784D66':
            row['id'], row['name'] = '20', 'Plum Noir 黑莓'
            
            
    elif row['brand'] == 'Giorgio Armani':
        if row['series'].startswith('絲絨啞亮唇釉'):
            row['series'] = '絲絨啞亮唇釉'
        elif row['series'].startswith('Giorgio Armani 絲滑啞亮唇膏'):
            row['series'] = '絲滑啞亮唇膏'
        elif row['series'].startswith('絲滑啞亮唇膏'):
            row['series'] = '絲滑啞亮唇膏'
        elif row['series'].startswith('阿玛尼唇膏102'):
            row['series'] = '唇膏102'
        elif row['series'].startswith('Giorgio Armani 漆亮水漾唇釉'):
            row['series'] = '漆亮水漾唇釉'
        elif row['series'].startswith('持色凝彩啞光染唇液'):
            row['series'] = '持色凝彩啞光染唇液'
        elif row['series'].startswith('Giorgio Armani 磁魅啞豔唇釉'):
            row['series'] = '磁魅啞豔唇釉'
        elif row['series'].startswith('瑩潤迷情唇膏'):
            row['series'] = '瑩潤迷情唇膏'
        elif row['series'].startswith('凝亮修護唇膏'):
            row['series'] = '凝亮修護唇膏' 
        elif row['series'].startswith('Giorgio Armani 絲絨啞亮唇釉'):
            row['series'] = '絲絨啞亮唇釉'
        elif row['series'].startswith('漆光迷情唇釉'):
            row['series'] = '漆光迷情唇釉'
            
            
    elif row['brand'] == 'Clinique':
        if row['series'].startswith('Clinique Clinique耀彩柔潤唇膏'):
            row['series'] = '耀彩柔潤唇膏'
        elif row['series'].startswith('水漾蜜糖翹唇筆'):
            row['series'] = '水漾蜜糖翹唇筆'
        elif row['series'].startswith('水漾晶凍翹唇筆'):
            row['series'] = '水漾晶凍翹唇筆'
        elif row['series'].startswith('耀彩亮澤唇蜜'):
            row['series'] = '耀彩亮澤唇蜜'
        elif row['series'].startswith('紐約普普絲絨唇釉'):
            row['series'] = '紐約普普絲絨唇釉'
        elif row['series'].startswith('啞緻耀彩柔潤唇膏'):
            row['series'] = '啞緻耀彩柔潤唇膏'
        elif row['series'].startswith('百變嘟嘟翹唇筆'):
            row['series'] = '百變嘟嘟翹唇筆'
        elif row['series'].startswith('持久唇膏'):
            row['series'] = '持久唇膏'
        elif row['series'].startswith('水漾甜心護唇凍'):
            row['series'] = '水漾甜心護唇凍'
        elif row['series'].startswith('沁甜恆久水唇蜜'):
            row['series'] = '沁甜恆久水唇蜜'
        elif row['series'].startswith('耀彩透亮唇膏'):
            row['series'] = '耀彩透亮唇膏'
            
            
    elif row['brand'] == 'Sisley':
        if row['series'].startswith('凝潤水感唇膏'):
            row['series'] = '凝潤水感唇膏'
        elif row['series'].startswith('植物閃漾唇彩筆'):
            row['series'] = '植物閃漾唇彩筆'
        elif row['series'].startswith('植物瑩亮唇膏'):
            row['series'] = '植物瑩亮唇膏'
            
    elif row['brand'] == 'Shiseido':
        if row['series'].startswith('激蜜輕吻唇膏'):
            row['series'] = '激蜜輕吻唇膏'
        elif row['series'].startswith('摩霧唇膏'):
            row['series'] = '摩霧唇膏'
        elif row['series'].startswith('激蜜光唇膏'):
            row['series'] = '激蜜光唇膏'
        elif row['series'].startswith('引釉光唇蜜'):
            row['series'] = '引釉光唇蜜'
        elif row['series'].startswith('臻红唇膏'):
            row['series'] = '臻红唇膏'
            
        row['id'] = row['id'].replace('#', '')
        if row['id'] == '518':
            row['color'] = '#eb3400'
            
            
    # 植村秀色号名字不对
    elif row['brand'] == 'Shu Uemura':
        if row['series'].startswith('豐盈啞緻唇膏'):
            row['series'] = '豐盈啞緻唇膏'
        elif row['series'].startswith('無色限唇膏'):
            row['series'] = '無色限唇膏'
        elif row['series'].startswith('無色限漆光唇釉'):
            row['series'] = '無色限漆光唇釉'
        elif row['series'].startswith('無色限柔霧唇釉'):
            row['series'] = '無色限柔霧唇釉'
        elif row['series'].startswith('無色限漆光唇膏'):
            row['series'] = '無色限漆光唇膏'
        elif row['series'].startswith('無色限柔霧唇膏'):
            row['series'] = '無色限柔霧唇膏'
        elif row['series'].startswith('無色限霓虹唇膏'):
            row['series'] = '無色限霓虹唇膏'
            
        if row['color'] == '#4935421380645_texture':
            row['color'], row['id'] = '#E14B4D', 'M OR 570'
            
    # 需要把前缀和色号一起改
    elif row['brand'] == 'Givenchy':
        if row['series'].startswith('Le Rose Perfecto潤唇膏'):
            row['series'] = 'Le Rose Perfecto 潤唇膏'
            
    elif row['brand'] == 'Estée Lauder':
        if row['series'].startswith('Estee Lauder Pure Color Love Lipstick'):
            row['series'] = '玩色戀愛唇膏'
        elif row['series'].startswith('Estee Lauder Pure Color Love 高清純色玩彩唇膏'):
            row['series'] = '高清純色玩彩唇膏'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy 花漾倾慕哑光唇膏'):
            row['series'] = '花漾倾慕哑光唇膏'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy 绝色倾慕液态唇膏 – 哑光'):
            row['series'] = '绝色倾慕液态唇膏 – 哑光'
        elif row['series'].startswith('Estee Lauder 雅詩蘭黛Pure Color Envy 傾慕唇膏魅色系列'):
            row['series'] = '傾慕唇膏魅色系列'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy 倾慕唇膏魅色系列'):
            row['series'] = '倾慕唇膏魅色系列'
        elif row['series'].startswith('Pure Color Desire 奢華慾望訂製唇膏'):
            row['series'] = '奢華慾望訂製唇膏'
        elif row['series'].startswith('Estee Lauder Pure Color Envy 高清純色魅惑唇膏'):
            row['series'] = '高清純色魅惑唇膏'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy 倾慕精油唇膏'):
            row['series'] = '倾慕精油唇膏'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy 绝色倾慕液态唇膏 – 金属'):
            row['series'] = '绝色倾慕液态唇膏 – 金属'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy 绝色倾慕液态唇膏 – 缎光'):
            row['series'] = '绝色倾慕液态唇膏 – 缎光'
        elif row['series'].startswith('Pure Color Envy 高清純色魅惑啞緻唇彩'):
            row['series'] = '高清純色魅惑啞緻唇彩'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy倾慕唇膏-丝绒系列'):
            row['series'] = '倾慕唇膏-丝绒系列'
        elif row['series'].startswith('雅诗兰黛Pure Color Envy花漾倾慕唇膏'):
            row['series'] = '花漾倾慕唇膏' 
        elif row['series'].startswith('Pure Color Envy 絕對慾望奢華潤唇膏'):
            row['series'] = '絕對慾望奢華潤唇膏'
        elif row['series'].startswith('Pure Color Envy 高清純色魅惑唇膏'):
            row['series'] = '高清純色魅惑唇膏'
        elif row['series'].startswith('雅诗兰黛Pure Color精油蜜唇膏'):
            row['series'] = '精油蜜唇膏'
        elif row['series'].startswith('Pure Color Envy Kissable Lip Shine'):
            row['series'] = 'Pure Color Envy Kissable Lip Shine'
        
        if row['color'] == '#bd0102':
            row['id'], row['name'] = '305', "Don't Stop"
        elif row['color'] == '#c0113c':
            row['name'] = 'Shock & Awe (Matte)'
        elif row['color'] == '#f87796':
            row['id'], row['name'] = '240', 'Pret-a-Party (Creme)'
        elif row['color'] == '#f16780':
            row['id'], row['name'] = '250', 'Radical Chic (Creme)'
        elif row['color'] == '#c6795b':
            row['id'], row['name'] = '140', 'Naked City (Creme)'
        elif row['color'] == '#bb6f82':
            row['id'], row['name'] = '430', 'Crazy Beautiful (Creme)'
        elif row['color'] == '#f8596b':
            row['id'], row['name'] = '330', 'Wild Poppy (Creme)'
        elif row['color'] == '#fd5147':
            row['id'], row['name'] = '340', 'Hot Rumor (Creme)'
        elif row['color'] == '#d05e9a':
            row['id'], row['name'] = '440', 'Hi-Voltage (Creme)'
        elif row['color'] == '#e63150':
            row['id'], row['name'] = '280', 'Haute & Cold (Creme)'
            
        if not row['name']:
            temp = row['id'].split(' ')
            row['id'], row['name'] = temp[0], ''.join(temp[1:])
            
        row['name'].replace('CrÃƒÂ¨me', 'Creme')
        row['name'].replace(u"(缺货)", u"")


    elif row['brand'] == 'Clarins':
        if row['series'].startswith('凝亮護唇油'):
            row['series'] = '凝亮護唇油'
        elif row['series'].startswith('啞緻豐盈唇膏'):
            row['series'] = '啞緻豐盈唇膏'
        elif row['series'].startswith('水潤豐盈唇膏'):
            row['series'] = '水潤豐盈唇膏'
        elif row['series'].startswith('瞬效豐盈唇蜜'):
            row['series'] = '瞬效豐盈唇蜜'
        elif row['series'].startswith('鎖色彩繪唇水'):
            row['series'] = '鎖色彩繪唇水'

        if not row['name']:
            temp = row['id'].split(' ')
            row['id'], row['name'] = temp[0], ''.join(temp[1:])
            
        if row['id'] == '760':
            row['name'] = 'Pink Cranberry'
            
    elif row['brand'] == 'Urban Decay':
        texture = re.findall('\((.*?)\)', row['series'])
        if texture:
            row['texture'] = texture[0]
        if row['series'].startswith('Urban Decay 24/7'):
            row['series'] = '24/7 唇筆'
        elif row['series'].startswith('Urban Decay Vice 唇膏'):
            row['series'] = 'Vice 唇膏'
        elif row['series'].startswith('Urban Decay 唇膏'):
            row['series'] = '唇膏'
        elif row['series'].startswith('VICE 唇釉'):
            row['series'] = 'Vice 唇釉'
        elif row['series'].startswith('VICE 特效疊色唇釉'):
            row['series'] = 'Vice 唇釉'
        elif row['series'].startswith('Vice 唇膏'):
            row['series'] = 'Vice 唇膏'
        elif row['series'].startswith('Urban Decay Vice唇膏'):
            row['series'] = 'Vice 唇膏'
            
    elif row['brand'] == 'MAKE UP FOR EVER':
        if row['series'].startswith('明星摯愛唇膏'):
            row['series'] = '明星摯愛唇膏'
        elif row['series'].startswith('明星摯愛持久啞光柔霧唇釉'):
            row['series'] = '明星摯愛持久啞光柔霧唇釉'
        elif row['series'].startswith('明星摯愛柔潤唇膏'):
            row['series'] = '明星摯愛柔潤唇膏'
        elif row['series'].startswith('清晰無痕妝前潤唇精華露'):
            row['series'] = '清晰無痕妝前潤唇精華露'
            
    elif row['brand'] == 'NARS':
        if row['series'].startswith('啞緻水彩唇釉'):
            row['series'] = '啞緻水彩唇釉'
        elif row['series'].startswith('時尚經典唇膏 (絲霧) '):
            row['series'] = '時尚經典唇膏 (絲霧) '
        elif row['series'].startswith('極緻奢華漆光唇釉'):
            row['series'] = '極緻奢華漆光唇釉'
        elif row['series'].startswith('絲絨柔滑唇釉'):
            row['series'] = '絲絨柔滑唇釉'
        elif row['series'].startswith('絲緞亮澤唇膏'):
            row['series'] = '絲緞亮澤唇膏'
        elif row['series'].startswith('絲絨亮彩唇膏筆'):
            row['series'] = '絲絨亮彩唇膏筆'
        
        if row['color'] == '#2784 POWERMATTE LIP PIGMENT - CALL ME (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Call Me', '', '#c46065'
        elif row['color'] == '#2780 POWERMATTE LIP PIGMENT - SOMEBODY TO LOVE (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Somebody to Love', '', '#ab5944'
        elif row['color'] == '#2787 POWERMATTE LIP PIGMENT - SPIN ME (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Spin Me', '', '#5c290f'
        elif row['color'] == '#2786 POWERMATTE LIP PIGMENT - VAIN (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Vain', '', '#b42a20'
        elif row['color'] == '#2734 VELVET LIP GLIDE - PARADISE GARAGE (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Paradise Garage', '', '#854948'
        elif row['color'] == '#2733 VELVET LIP GLIDE - ROSELAND (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Roseland', '', '#ba696d'
        elif row['color'] == '#2732 VELVET LIP GLIDE - SWING (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Swing', '', '#bb7764'
        elif row['color'] == '#2731 VELVET LIP GLIDE - XENON (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Xenon', '', '#b66760'
        elif row['color'] == '#9408 SATIN LIPSTICK - ORGASM (SWATCH).jpg':
            row['id'], row['name'], row['color'] = 'Orgasm', '', '#CF8978'
            
    elif row['brand'] == 'By Terry':
        if row['series'].startswith('大師絲絨霧面口紅'):
            row['series'] = '大師絲絨霧面口紅'
        elif row['series'].startswith('大師閃亮滋潤口紅'):
            row['series'] = '大師閃亮滋潤口紅'
        elif row['series'].startswith('驚艷絲絨唇膏筆細管按壓口紅'):
            row['series'] = '驚艷絲絨唇膏筆細管按壓口紅'
    
        if row['color'] == '#9':
            row['color'] = '#b50021'
    
    elif row['brand'] == 'Anastasia Beverly Hills':
        if row['series'].startswith('Lip Gloss'):
            row['series'] = 'Lip Gloss'
        elif row['series'].startswith('Liquid Lipstick'):
            row['series'] = 'Liquid Lipstick'
        elif row['series'].startswith('哑光唇膏'):
            row['series'] = '哑光唇膏'
        elif row['series'].startswith('唇彩'):
            row['series'] = '唇彩'
        elif row['series'].startswith('啞光唇膏'):
            row['series'] = '啞光唇膏'
        elif row['series'].startswith('唇蜜'):
            row['series'] = '唇蜜'
            
        if row['id'] == 'c56c66':
            row['id'], row['name'], row['color'] = 'Vintage', '', '#c56c66'
        elif row['id'] == 'b35e4a':
            row['id'], row['name'], row['color'] = 'Caramel', '', '#b35e4a'
        elif row['id'] == 'ffaba1':
            row['id'], row['name'], row['color'] = 'Girly', '', '#ffaba1'
        elif row['id'] == 'a96564':
            row['id'], row['name'], row['color'] = 'Kristen', '', '#a96564'
        elif row['id'] == 'e7556d':
            row['id'], row['name'], row['color'] = 'Metallic Rose', '', '#e7556d'
        elif row['id'] == '924a4d':
            row['id'], row['name'], row['color'] = 'Parfait', '', '#924a4d'
        elif row['color'] == '#CF9182':
            row['id'], row['name'] = 'Toffee', ''
    
    elif row['brand'] == 'Laura Mercier':
        if row['series'].startswith('Lip Parfait Creamy漾彩唇膏'):
            row['series'], row['texture'] = '漾彩唇膏', 'creamy'
        elif row['series'].startswith('Paint Wash'):
            row['series'] = '唇彩'
        elif row['series'].startswith('Velour臻至啞光唇膏'):
            row['series'], row['texture'] = 'Velour臻至啞光唇膏', '啞光'
        elif row['series'].startswith('唇線筆') or row['series'].startswith('唇线笔'):
            row['series'] = '唇線筆'
        elif row['series'].startswith('液體唇膏'):
            row['series'] = '唇彩'
            
    elif row['brand'] == 'Hourglass':
        if row['series'].startswith('GIRL 唇膏筆'):
            row['series'] = 'GIRL 唇膏筆'
        elif row['series'].startswith('Nº 28 唇部護理油'):
            row['series'] = 'Nº 28 唇部護理油'
        elif row['series'].startswith('全方位長效唇線筆'):
            row['series'] = '全方位長效唇線筆'
        elif row['series'].startswith('告解蜜語高飽和美形唇膏'):
            row['series'] = '告解蜜語高飽和美形唇膏'
        elif row['series'].startswith('絢彩絲絨液體唇膏'):
            row['series'] = '絢彩絲絨液體唇膏'
        elif row['series'].startswith('豐潤閃亮唇彩'):
            row['series'] = '豐潤閃亮唇彩'
        elif row['series'].startswith('魅惑裸色蠟筆唇膏'):
            row['series'] = '魅惑裸色蠟筆唇膏'
            
        if row['color'] == '#883A36':
            row['id'], row['name'] = "I've been", ''
        elif row['color'] == '#98272D':
            row['id'] = "I Can't Live Without"
        elif row['color'] == '#AB004F':
            row['id'] = "I Can't Wait"
        elif row['color'] == '#9F6951':
            row['id'] = "I'll Never Stop"
        elif row['color'] == '#AA483F':
            row['id'] = "I'm Addicted"
        elif row['color'] == '#B66C79':
            row['id'] = "I've Kissed"
        elif row['color'] == '#A75348':
            row['id'] = "I've Never"
        elif row['id'] == "When I%22m With You":
            row['id'], row['color'] = "When I'm With You", '#ca3279'
        elif row['color'] == '#942B49':
            row['id'], row['color'] = "When I'm Alone", "#ae2f5e"
            
    elif row['brand'] == 'Gucci':
        if row['series'].startswith('傾色絲潤唇膏'):
            row['series'] = '傾色絲潤唇膏'
            
    elif row['brand'] == 'Tom Ford':
        if row['series'].startswith('激情幻魅唇膏'):
            row['series'] = '激情幻魅唇膏'
            
    elif row['brand'] == "L'Oréal":
        if row['series'].startswith('紛澤琉金唇膏'):
            row['series'] = '紛澤琉金唇膏'
            
    elif row['brand'] == 'Benefit Cosmetics':
        if row['series'].startswith('TheyRe Real!雙色唇膏'):
            row['series'] = "They're Real!雙色唇膏"

    elif row['brand'] == 'Guerlain':
        if row['series'].startswith("Rouge G"):
            row['series'] = "Rouge G 寶石唇膏"         
    # TODO: 去掉“缺货”

In [120]:
df = pd.DataFrame(result)
df['Name'] = df['id'] + ' ' + df['name']
df.drop([
    625, 631, 958
], inplace=True)
df[
    df['brand'] == "NARS"
].sort_values(['series', 'Name'])[['brand', 'series', 'price', 'color', 'Name', 'texture', 'id', 'name', 'link']]

,brand,series,price,color,Name,texture,id,name,link
506,NARS,啞緻水彩唇釉,HKD 250.00,#c46065,Call Me,NaN,Call Me,,https://www.dfs.com/tc/hong-kong/brands/nars/44686780/powermatte-lip-pigment---call-me-55ml
969,NARS,啞緻水彩唇釉,HKD 250.00,#FF5E63,Firecracker(缺货),NaN,Firecracker(缺货),,https://www.dfs.com/tc/hong-kong/brands/nars/60004984/powermatte-lip-pigment---firecracker-55ml
968,NARS,啞緻水彩唇釉,HKD 250.00,#E35F6C,Firework(缺货),NaN,Firework(缺货),,https://www.dfs.com/tc/hong-kong/brands/nars/60004985/powermatte-lip-pigment---firework-55ml
967,NARS,啞緻水彩唇釉,HKD 250.00,#CF6E5E,Flame(缺货),NaN,Flame(缺货),,https://www.dfs.com/tc/hong-kong/brands/nars/60004982/powermatte-lip-pigment---flame-55ml
492,NARS,啞緻水彩唇釉,HKD 250.00,#ab5944,Somebody to Love,NaN,Somebody to Love,,https://www.dfs.com/tc/hong-kong/brands/nars/44686772/powermatte-lip-pigment---somebody-to-love-55ml
505,NARS,啞緻水彩唇釉,HKD 250.00,#5c290f,Spin Me,NaN,Spin Me,,https://www.dfs.com/tc/hong-kong/brands/nars/44686814/powermatte-lip-pigment---spin-me-55ml
947,NARS,啞緻水彩唇釉,HKD 250.00,#b42a20,Vain,NaN,Vain,,https://www.dfs.com/tc/hong-kong/brands/nars/44686806/powermatte-lip-pigment---vain-55ml
850,NARS,時尚經典唇膏 (絲霧),HKD 250.00,#C92121,Red Lizard(缺货),NaN,Red Lizard(缺货),,https://www.dfs.com/tc/hong-kong/brands/nars/39236377/semi-matte-lipstick---red-lizard-34g
849,NARS,時尚經典唇膏 (絲霧),HKD 250.00,#F741A4,Schiap(缺货),NaN,Schiap(缺货),,https://www.dfs.com/tc/hong-kong/brands/nars/39236419/semi-matte-lipstick---schiap-34g
948,NARS,極緻奢華漆光唇釉,HKD 250.00,#E06B7D,Abruzzo(缺货),NaN,Abruzzo(缺货),,https://www.dfs.com/tc/hong-kong/brands/nars/44686681/vinyl-lip-lacquer---abruzzo-55ml


In [126]:
df['brand'].value_counts()

Bobbi Brown                134
Hourglass                   85
Estée Lauder                76
SAINT LAURENT               71
Dior                        66
Clinique                    63
Shiseido                    61
Lancôme                     57
Urban Decay                 47
Giorgio Armani              41
Laura Mercier               40
Anastasia Beverly Hills     37
Sisley                      36
MAKE UP FOR EVER            33
Clé de Peau Beauté          31
Guerlain                    29
Clarins                     27
Shu Uemura                  27
NARS                        16
Givenchy                    14
By Terry                    11
The History of Whoo          9
Tom Ford                     5
Benefit Cosmetics            4
L'Oréal                      1
Gucci                        1
Name: brand, dtype: int64

In [ ]:
#     elif row['brand'] == 'Clarins':
#         if row['series'].startswith():
#             row['series'] = 
#         elif row['series'].startswith():
#             row['series'] =
#         elif row['series'].startswith():
#             row['series'] = 
#         elif row['series'].startswith():
#             row['series'] =
#         elif row['series'].startswith():
#             row['series'] =
#         elif row['series'].startswith():
#             row['series'] = 
#         elif row['series'].startswith():
#             row['series'] =
        
#         if row['color'] == '':
#             row['id'], row['name'] = ,
#         elif row['color'] == '':
#             row['id'], row['name'] = ,
#         elif row['color'] == '':
#             row['id'], row['name'] = ,
#         elif row['color'] == '':
#             row['id'], row['name'] = ,
#         elif row['color'] == '':
#             row['id'], row['name'] = ,
#         elif row['color'] == '':
#             row['id'], row['name'] = ,

In [148]:
df[
    ~df['texture'].isnull()
].sort_values(['series'])[['brand', 'series', 'price', 'color', 'Name', 'texture', 'id', 'name', 'link']]

,brand,series,price,color,Name,texture,id,name,link
668,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#EB5343,Stylin,啞光,Stylin,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020466/velour-extreme-matte-lipstick---stylin-14g
664,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#CC368F,Goal,啞光,Goal,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020503/velour-extreme-matte-lipstick---goal-14g
667,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#95211A,Control,啞光,Control,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020457/velour-extreme-matte-lipstick---control-14g
985,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#AB2477,Hot(缺货),啞光,Hot(缺货),,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020686/velour-extreme-matte-lipstick---hot-14g
669,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#B16B76,Fresh,啞光,Fresh,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020440/velour-extreme-matte-lipstick---fresh-14g
671,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#E00C65,It Girl,啞光,It Girl,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020432/velour-extreme-matte-lipstick---it-girl-14g
672,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#C80463,Fab,啞光,Fab,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020460/velour-extreme-matte-lipstick---fab-14g
674,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#D2233D,Dominate,啞光,Dominate,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020415/velour-extreme-matte-lipstick---dominate-14g
676,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#6A1C3B,Fatale,啞光,Fatale,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020428/velour-extreme-matte-lipstick---fatale-14g
677,Laura Mercier,Velour臻至啞光唇膏,HKD 260.00,#F21528,Fire,啞光,Fire,,https://www.dfs.com/tc/hong-kong/brands/laura-mercier/60020519/velour-extreme-matte-lipstick---fire-14g


In [123]:
df[
    df.duplicated(['color'])
].sort_values(['color'])

,brand,series,price,color,id,name,link,texture,Name
36,Lancôme,絕對完美鑽石霧光唇膏,HKD 270.00,#0,#02,,https://www.dfs.com/tc/hong-kong/brands/lancome/60125607/lancome-absolu-rouge-ruby-cream-02-3g,NaN,#02
43,Lancôme,絕對完美鑽石霧光唇膏,HKD 270.00,#0,#03,,https://www.dfs.com/tc/hong-kong/brands/lancome/60125605/lancome-absolu-rouge-ruby-cream-03-3g,NaN,#03
111,Lancôme,絕對完美鑽石霧光唇膏,HKD 270.00,#0,#481,,https://www.dfs.com/tc/hong-kong/brands/lancome/60125604/lancome-absolu-rouge-ruby-cream-481-3g,NaN,#481
251,Lancôme,蘭蔻菁純柔潤脣膏 (絲絨),HKD 270.00,#0,#373,,https://www.dfs.com/tc/hong-kong/brands/lancome/40274615/labsolu-rouge-matte-198-34g,NaN,#373
252,Lancôme,蘭蔻菁純柔潤脣膏 (絲絨),HKD 270.00,#0,#192,,https://www.dfs.com/tc/hong-kong/brands/lancome/40274615/labsolu-rouge-matte-198-34g,NaN,#192
253,Lancôme,蘭蔻菁純柔潤脣膏 (絲絨),HKD 270.00,#0,#170,,https://www.dfs.com/tc/hong-kong/brands/lancome/40274615/labsolu-rouge-matte-198-34g,NaN,#170
254,Lancôme,蘭蔻菁純柔潤脣膏 (絲絨),HKD 270.00,#0,#00,,https://www.dfs.com/tc/hong-kong/brands/lancome/40274615/labsolu-rouge-matte-198-34g,NaN,#00
314,Lancôme,"蘭蔻菁純柔潤唇釉 三支裝 #132, #317, #378 8ml*3",HKD 675.00,#0,"#132, #317, #378",,https://www.dfs.com/tc/hong-kong/brands/lancome/42131524/lancome-l-absolu-gloss-trio-132-317-378-8ml3,NaN,"#132, #317, #378"
316,Lancôme,瑰麗豐盈唇膏液,HKD 230.00,#0,#422,,https://www.dfs.com/tc/hong-kong/brands/lancome/42131490/labsolu-gloss-cream-382-8ml,NaN,#422
761,Lancôme,菁純寶石唇膏,HKD 270.00,#0,#01,,https://www.dfs.com/tc/hong-kong/brands/lancome/60125601/lancome-absolu-rouge-ruby-cream-01-3g,NaN,#01


# Olivia

In [141]:
olivia = None
with open('../lipstick/src/lipstick.json') as f:
    olivia = json.load(f)
    
olivia_rows = []
for brand in olivia['brands']:
    for series in brand['series']:
        for lipstick in series['lipsticks']:
            olivia_rows.append({
                'brand': brand['name'],
                'series': series['name'],
                'color': lipstick['color'],
                'id': lipstick['id'],
                'name': lipstick['name']
            })
            
df_old = pd.DataFrame(olivia_rows)

In [333]:
Counter(df_old['brand'])

Counter({'圣罗兰': 109, '香奈儿可可小姐': 60, '迪奥': 66, '美宝莲': 6, '纪梵希': 46})

In [144]:
print(len(df_old))
df_old['brand'].value_counts()

287


圣罗兰        109
迪奥          66
香奈儿可可小姐     60
纪梵希         46
美宝莲          6
Name: brand, dtype: int64

In [334]:
len(df_old)

287

https://en.stylenanda.com/category/3ce/1784/